<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Optimizing-specific-fluxonium-property" data-toc-modified-id="Optimizing-specific-fluxonium-property-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Optimizing specific fluxonium property</a></span><ul class="toc-item"><li><span><a href="#Frequencies-and-matrix-elements-variation" data-toc-modified-id="Frequencies-and-matrix-elements-variation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Frequencies and matrix elements variation</a></span></li><li><span><a href="#Relaxation-time-at-sweet-spot" data-toc-modified-id="Relaxation-time-at-sweet-spot-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Relaxation time at sweet spot</a></span></li><li><span><a href="#Frequency-fluctuation" data-toc-modified-id="Frequency-fluctuation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Frequency fluctuation</a></span></li><li><span><a href="#Wavefunctions" data-toc-modified-id="Wavefunctions-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Wavefunctions</a></span></li></ul></li><li><span><a href="#Sweeping-qubit-parameters-for-T1-and-T2" data-toc-modified-id="Sweeping-qubit-parameters-for-T1-and-T2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Sweeping qubit parameters for T1 and T2</a></span><ul class="toc-item"><li><span><a href="#Sweep-EJ-and-EL-for-T1-due-to-dielectric-loss" data-toc-modified-id="Sweep-EJ-and-EL-for-T1-due-to-dielectric-loss-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Sweep EJ and EL for T1 due to dielectric loss</a></span></li><li><span><a href="#Sweep-EJ-and-EL-for-T2-due-to-flux-noise" data-toc-modified-id="Sweep-EJ-and-EL-for-T2-due-to-flux-noise-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Sweep EJ and EL for T2 due to flux noise</a></span></li></ul></li></ul></div>

# Optimizing specific fluxonium property


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from qutip import*
import sys
sys.path.append('/Users/longnguyen/Documents/GitHub/Fluxonium_berkeley/')
from Fluxonium_hamiltonians import Single_small_junction as fluxonium
import plotting_settings
from scipy.optimize import minimize
from scipy.special import eval_hermite as hpoly
import plotting_settings
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 400
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']

In [ ]:
Q = 2*np.pi*5e9 * 500e-6
print (Q*1e-6)

## Frequencies and matrix elements variation

In [ ]:
N = 30 #number of levels
E_C = 1 #Charging energy, GHz
level_num = 10
phi_ext = 0.5

h = 6.626e-34 #Planck constant
kB = 1.38e-23 #Boltzmann constant

In [ ]:
E_L = np.linspace (0.4,1.4,101)
E_J = np.linspace(3.5,3.5,1)
energies = np.zeros((len(E_J), len(E_L), 10))
# n_me = np.zeros((len(E_J), len(E_L), 10))
for idx, E_J_temp in enumerate(E_J):
    for idy, E_L_temp in enumerate(E_L):
        H = fluxonium.bare_hamiltonian(N, E_L_temp, E_C, E_J_temp, phi_ext*2*np.pi)
        energies[idx, idy,:] = H.eigenenergies()[:10]
#         n_me[idx, idy, 0] = fluxonium.charge_matrix_element(N, E_L_temp, E_C, E_J_temp, phi_ex*2*np.pi, 0, 1)
#         n_me[idx, idy, 1] = fluxonium.charge_matrix_element(N, E_L_temp, E_C, E_J_temp, phi*2*np.pi, 1, 2)

In [ ]:
plt.plot()

In [ ]:
#Lowest energy
trans_01 = energies[:, :,1] - energies[:, :,0]
X,Y = np.meshgrid(E_L,E_J)
Z = trans_01
plt.pcolormesh(X,Y,Z, cmap = 'RdBu_r')#, vmin = 0.2, vmax = 1.5)
plt.ylabel(r'$E_J~(\mathrm{GHz})$')
plt.xlabel(r'$E_L~(\mathrm{GHz})$')
plt.colorbar()

In [ ]:
#anharmonicity
trans_12 = energies[:, :,2] - energies[:, :,1]
anhar = trans_12-trans_01
X,Y = np.meshgrid(E_L,E_J)
Z = anhar
plt.pcolormesh(X,Y,Z, cmap = 'RdBu_r')
plt.ylabel(r'$E_J~\mathrm{(GHz)}$')
plt.xlabel(r'$E_L~\mathrm{(GHz)}$')
plt.colorbar()

In [ ]:
#01 matrix element
X,Y = np.meshgrid(E_L,E_J)
Z = n_me[:,:,0]
plt.pcolormesh(X,Y,Z, cmap = 'RdBu_r')
plt.ylabel(r'$E_J~\mathrm{(GHz)}$')
plt.xlabel(r'$E_L~\mathrm{(GHz)}$')
plt.colorbar()

In [ ]:
#12 matrix element
X,Y = np.meshgrid(E_L,E_J)
Z = n_me[:,:,1]
plt.pcolormesh(X,Y,Z, cmap = 'RdBu_r')
plt.ylabel(r'$E_J~\mathrm{(GHz)}$')
plt.xlabel(r'$E_L~\mathrm{(GHz)}$')
plt.colorbar()

## Relaxation time at sweet spot

In [ ]:
T_diel = 20e-3 #effective temperature in K
T_qp = 20e-4 #effective temperature in K
Q_cap = (2e-7)**-1
Q_qp = (2e-9)**-1
phi = 0.5
print (1/Q_cap)
print (1/Q_qp)

In [ ]:
def gamma1(x):
    E_L = x[0]
    E_J = x[1]
    H = fluxonium.bare_hamiltonian(N, E_L, E_C, E_J, phi*2*np.pi) #Hamiltonian
    freq = H.eigenenergies()[1] - H.eigenenergies()[0] #transition freq
    thermal_factor_diel = (1+np.exp(-h*freq*1e9/(kB*T_diel)))
    thermal_factor_qp = (1+np.exp(-h*freq*1e9/(kB*T_qp)))
    p_me = fluxonium.phase_matrix_element(N, E_L, E_C, E_J, phi*2*np.pi, 0, 1) #Matrix element
    gamma_cap = fluxonium.relaxation_rate_cap(E_L, E_C, E_J, Q_cap,freq, p_me, T_diel)*thermal_factor_diel
    gamma_qp = fluxonium.relaxation_rate_qp_array(E_L, E_C, E_J, Q_qp, freq, p_me)*thermal_factor_qp
    return abs(p_me)

In [ ]:
# x0 = [1.0,4.0]
# opt = minimize(gamma1, x0, method='Powell', tol = 1e-1)
# print (opt)

In [ ]:
E_L_array = np.linspace(0.2,1.6,71)
E_J_array = np.linspace(1,8,71)
gamma = np.zeros((len(E_J_array), len(E_L_array), 2), dtype = complex)
def gamma1(E_L, E_J):
    H = fluxonium.bare_hamiltonian(N, E_L, E_C, E_J, phi*2*np.pi) #Hamiltonian
    freq = H.eigenenergies()[1] - H.eigenenergies()[0] #transition freq
    thermal_factor_diel = (1+np.exp(-h*freq*1e9/(kB*T_diel)))
    thermal_factor_qp = (1+np.exp(-h*freq*1e9/(kB*T_qp)))
    p_me = fluxonium.phase_matrix_element(N, E_L, E_C, E_J, phi*2*np.pi, 0, 1) #Matrix element
    gamma_cap = fluxonium.relaxation_rate_cap(E_L, E_C, E_J, Q_cap,freq, p_me, T_diel)*thermal_factor_diel
    gamma_qp = fluxonium.relaxation_rate_qp_array(E_L, E_C, E_J, Q_qp, freq, p_me)*thermal_factor_qp
    return gamma_cap, gamma_qp

for J_idx, E_J in enumerate(E_J_array):
    for L_idx, E_L in enumerate(E_L_array):
        gamma[J_idx, L_idx, :] = gamma1(E_L, E_J)

In [ ]:
plt.figure(figsize=[10,10])
import matplotlib.colors as cl
plt.xlabel(r'$E_L$')
plt.ylabel(r'$E_J$')
X,Y = np.meshgrid(E_L_array, E_J_array)
Z = abs(gamma[:,:,0])
plt.pcolormesh(X,Y,(Z**-1)*1e6,norm=cl.LogNorm())
plt.colorbar()

In [ ]:
plt.figure(figsize=[10,10])
plt.xlabel(r'$E_L$')
plt.ylabel(r'$E_J$')
X,Y = np.meshgrid(E_L_array, E_J_array)
Z = abs(gamma[:,:,1])
plt.pcolormesh(X,Y,(Z**-1)*1e6,norm=cl.LogNorm())
plt.colorbar()

In [ ]:
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']
figpath = '/Users/longnguyen/Google Drive/LBL Research/Illustration/HCF paper/'
figname = 'fluxonium_T1_sweep1.pdf'

plt.figure(figsize=[5,4])

plt.ylabel(r'$E_L$ (GHz)')
plt.xlabel(r'$E_J$ (GHz)')
X,Y = np.meshgrid(E_J_array, E_L_array)
Z = (abs(gamma[:,:,1])+abs(gamma[:,:,0])).transpose()
# Z = abs(gamma[:,:,0]).transpose()
# plt.pcolormesh(X,Y,(Z**-1)*1e6, cmap = 'GnBu',norm=cl.LogNorm(),vmin = 1e2, vmax = 1e3)
plt.pcolormesh(X,Y,(Z**-1)*1e6, cmap = 'GnBu', vmin = 1e2, vmax = 1e3)
cbar=plt.colorbar()
# cbar.set_ticks([1e2, 1e3])
# cbar.set_ticklabels([1e2, 1e3])
contours = plt.contour(X,Y,(Z**-1)*1e6,10, colors = 'black', vmin = 1e2, vmax = 1e3,linewidths = 0.7, alpha = 1)
plt.clabel(contours, inline=True, fontsize=10, fmt='%1.0f')
plt.yticks([0.4, 0.8, 1.2, 1.6])
plt.xticks([2,4,6,8])

plt.tight_layout()
# plt.savefig(figpath+figname)

## Frequency fluctuation

In [ ]:
N = 30 #number of levels
E_J = 4 #Josephson energy, GHz
E_C = 2 #Charging energy, GHz
E_L = 1 #Inductive energy, GHz
level_num = 10
phi_ext = 0.5 

H = fluxonium.bare_hamiltonian(N, E_L, E_C, E_J, np.pi)
energies = H.eigenenergies()
anchor = energies[1] - energies[0]

In [ ]:
dE_L = E_L*0.1
dE_J = E_J*0.1
#new EJ
E_J_fluc = np.linspace(E_J-dE_J, E_J+dE_J, 101)
E_L_fluc = np.linspace(E_L-dE_L, E_L+dE_L, 101)
trans_freq = np.zeros((len(E_J_fluc), len(E_L_fluc)))
for idx, E_J_temp in enumerate(E_J_fluc):
    for idy, E_L_temp in enumerate(E_L_fluc):
        H = fluxonium.bare_hamiltonian(N, E_L_temp, E_C, E_J_temp, phi_ext*2*np.pi)
        energies = H.eigenenergies()
        trans_freq[idx, idy] = energies[1] - energies[0] - anchor
        

In [ ]:
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']
figpath = '/Users/longnguyen/Google Drive/LBL Research/Illustration/HCF paper/'
figname = 'fluxonium_spectrum_sweep.pdf'


plt.figure(figsize = [5.6,4])
X,Y = np.meshgrid((E_J_fluc-E_J)/E_J*100, (E_L_fluc-E_L)/E_L*100)
Z = (trans_freq*1e3).transpose()
plt.pcolor(X, Y, Z, cmap = 'RdBu_r', vmax = 200, vmin = -200)
clb = plt.colorbar()
# clb.ax.set_title(r'$\delta \omega_{01}/2\pi~(\mathrm{MHz})$')

# cbar=plt.colorbar()
# cbar.set_ticks([1e2, 1e3])
# cbar.set_ticklabels([1e2, 1e3])
contours = plt.contour(X,Y,Z,10, colors = 'black',linewidths = 0.7, alpha = 1)
plt.clabel(contours, inline=True, fontsize=10, fmt='%1.0f')


# plt.axhline(y=0, linestyle = '--', linewidth = 1.0, alpha = 0.5, color = 'C0')
# plt.axvline(x=0, linestyle = '--',linewidth = 1.0, alpha = 0.5, color = 'C1')
# plt.grid()
plt.xlabel(r'$\delta E_J/E_J (\%)$')
plt.ylabel(r'$\delta E_L/E_L (\%)$')
plt.tight_layout()
plt.savefig(figpath+figname)

In [ ]:
plt.figure(figsize = [5,4])
plt.plot((E_J_fluc-E_J)/E_J*100, trans_freq[:,50]*1e3, linestyle = '-', label = r'$E_J$ fluctuation')
plt.plot((E_J_fluc-E_J)/E_J*100, trans_freq[50,:]*1e3,linestyle = '--', label = r'$E_L$ fluctuation')
plt.xlabel(r'$\delta E_i/E_i (\%)$')
plt.ylabel(r'$\delta \omega_{01}/2\pi~(\mathrm{MHz})$')
plt.xlim([-10,10])
plt.ylim([-140,140])
plt.legend(fontsize =12)
plt.grid()
plt.tight_layout()
figname = 'fluxonium_freq_dispersion1.pdf'
plt.savefig(figpath+figname)

In [ ]:
N = 30 #number of levels
E_J = 3.5 #Josephson energy, GHz
E_C = 1 #Charging energy, GHz
E_L = 0.7 #Inductive energy, GHz
level_num = 10
phi_ext = 0.5 

dE_L = E_L*0.1
dE_J = E_J*0.1
#new EJ
E_J_fluc = np.linspace(E_J-dE_J, E_J+dE_J, 101)
E_L_fluc = np.linspace(E_L-dE_L, E_L+dE_L, 101)
trans_freq_EL = np.zeros((len(E_L_fluc), 3))
trans_freq_EJ = np.zeros((len(E_J_fluc), 3))

for idx, E_L_temp in enumerate(E_L_fluc):
    H = fluxonium.bare_hamiltonian(N, E_L_temp, E_C, E_J, phi_ext*2*np.pi)
    energies = H.eigenenergies()
    trans_freq_EL[idx, 0] = energies[3] - energies[0] 
    trans_freq_EL[idx, 1] = energies[4] - energies[1]
    trans_freq_EL[idx, 2] = energies[1] - energies[0]




for idx, E_J_temp in enumerate(E_J_fluc):
    H = fluxonium.bare_hamiltonian(N, E_L, E_C, E_J_temp, phi_ext*2*np.pi)
    energies = H.eigenenergies()
    trans_freq_EJ[idx, 0] = energies[3] - energies[0] 
    trans_freq_EJ[idx, 1] = energies[4] - energies[1]
    trans_freq_EJ[idx, 2] = energies[1] - energies[0]
    

In [ ]:
plt.figure(figsize=[6,3])
plt.plot(trans_freq_EL[:, 0], (E_L_fluc-E_L)/E_L*100)
plt.plot(trans_freq_EL[:, 1], (E_L_fluc-E_L)/E_L*100)

plt.plot(trans_freq_EJ[:, 0], (E_J_fluc-E_J)/E_J*100, '--')
plt.plot(trans_freq_EJ[:, 1], (E_J_fluc-E_J)/E_J*100,'--',color='C4')

plt.ylabel(r'$\delta E_i/E_i (\%)$')
plt.xlabel("Frequency (GHz)")
# plt.plot(E_L_fluc, trans_freq_EL[:, 2])
plt.grid()
plt.ylim([(E_L_fluc[0]-E_L)/E_L*100,(E_L_fluc[-1]-E_L)/E_L*100])
plt.xlim([4,10])


## Wavefunctions

In [ ]:
N = 30 #number of levels
E_J = 3 #Josephson energy, GHz
E_C = 3 #Charging energy, GHz
E_L = 0.5 #Inductive energy, GHz
level_num = 10
phi_ext = np.linspace(0,1,101)
energies = np.zeros((len(phi_ext), level_num))
states = np.zeros((len(phi_ext), level_num, N))
max_display_y = 10

In [ ]:
for idx, phi in enumerate(phi_ext):
    H = fluxonium.bare_hamiltonian(N, E_L, E_C, E_J, phi*2*np.pi)
    energies[idx, :] = H.eigenenergies()[:level_num]

In [ ]:
def ho_wf(phi,l,Ec,El):
    ratio = (8.0*Ec/El)**(0.25)
    coeff = (2.0**l*np.math.factorial(l)*np.sqrt(np.pi)*ratio)**(-0.5)
    return coeff * np.exp(-0.5*(phi/ratio)**2) * hpoly(l,phi/ratio)

toPlotNum = 5
# phi_ext=[0.5]
phi = np.linspace(-3*np.pi,3*np.pi,101)
waveFunc = np.zeros((len(phi), len(phi_ext), toPlotNum))
potential = np.zeros((len(phi), len(phi_ext)))
for phi_idx, phi_e in enumerate(phi_ext):
    H = fluxonium.bare_hamiltonian(N,E_L,E_C,E_J, phi_e*2*np.pi)
    evalues, evectors = H.eigenstates()
    for state_idx in range(toPlotNum):
        wFunc = np.zeros(len(phi))
        for lvl_idx in range(N):
            coeff = np.real(evectors[state_idx].full()[lvl_idx, 0])
            wFunc = wFunc + coeff*ho_wf(phi, lvl_idx, E_C, E_L)
        waveFunc[:, phi_idx, state_idx] = wFunc + evalues[state_idx]
    potential[:,phi_idx] = 0.5*E_L*phi**2 - E_J*np.cos(phi-phi_e*2*np.pi) 

In [ ]:
plt.figure(figsize = [7,7])  
path = "/Users/longnguyen/Documents/wFunction1_"
for phi_idx in range(len(phi_ext)):
    fname = path +str(phi_ext)+'.png'
    plt.plot(phi,potential[:, phi_idx], linewidth = 5, color = 'k')
    for state_idx in range(toPlotNum):
        plt.plot(phi,np.ones(len(phi))*energies[phi_idx,state_idx], linewidth = 1, alpha = 0.5, color = 'g')
        plt.plot(phi,waveFunc[:, phi_idx, state_idx])
    plt.savefig(fname)

# Sweeping qubit parameters for T1 and T2 

## Sweep EJ and EL for T1 due to dielectric loss

In [ ]:
E_J_array = np.linspace(5,7,5)
E_L_array = np.linspace(0.2,0.8,5)
# print(E_L_array)
N = 30 #number of levels
E_J = 6 #Josephson energy, GHz
E_C = 1 #Charging energy, GHz
E_L = 0.5 #Inductive energy, GHz
level_num = 10
phi_ext = np.linspace(0,0.5,101)

#Sweep EJ
for idx, phi in enumerate(phi_ext):
    H = fluxonium.bare_hamiltonian(N, E_L, E_C, E_J, phi*2*np.pi)
    energy = H.eigenenergies()[:level_num]

## Sweep EJ and EL for T2 due to flux noise

In [ ]:
E_J_array = np.linspace(5,7,5)
E_L_array = np.linspace(0.2,0.8,5)
# print(E_L_array)
N = 30 #number of levels
E_J = 6 #Josephson energy, GHz
E_C = 1 #Charging energy, GHz
E_L = 0.5 #Inductive energy, GHz
level_num = 10
phi_ext = np.linspace(0,0.5,101)

